# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846906713553                   -0.70    4.5         
  2   -7.852323144214       -2.27       -1.53    1.0   22.9ms
  3   -7.852615824866       -3.53       -2.56    1.5   24.9ms
  4   -7.852646012064       -4.52       -2.89    2.5   31.5ms
  5   -7.852646522023       -6.29       -3.20    1.2   24.1ms
  6   -7.852646679449       -6.80       -4.07    1.0   22.7ms
  7   -7.852646686457       -8.15       -5.02    2.0   28.8ms
  8   -7.852646686723       -9.57       -5.43    1.8   27.4ms
  9   -7.852646686729      -11.26       -5.76    1.2   24.3ms
 10   -7.852646686730      -11.91       -6.53    1.0   23.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846802413515                   -0.70           4.8         
  2   -7.852526501659       -2.24       -1.63   0.80    2.0    256ms
  3   -7.852636819851       -3.96       -2.72   0.80    1.0   21.0ms
  4   -7.852646547804       -5.01       -3.26   0.80    2.0   27.2ms
  5   -7.852646678210       -6.88       -4.12   0.80    1.2   22.0ms
  6   -7.852646686553       -8.08       -4.84   0.80    2.0   27.3ms
  7   -7.852646686721       -9.78       -5.77   0.80    1.5   23.1ms
  8   -7.852646686730      -11.04       -6.85   0.80    2.2   27.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.391654e+01     3.432355e+00
 * time: 0.05856919288635254
     1     1.391283e+00     1.866601e+00
 * time: 0.2756969928741455
     2    -1.673300e+00     2.059787e+00
 * time: 0.3703281879425049
     3    -3.808010e+00     1.727641e+00
 * time: 0.40346503257751465
     4    -5.241508e+00     1.530867e+00
 * time: 0.4359281063079834
     5    -6.961766e+00     8.322097e-01
 * time: 0.4686281681060791
     6    -7.002847e+00     1.395273e+00
 * time: 0.49106502532958984
     7    -7.537092e+00     9.246721e-01
 * time: 0.5137879848480225
     8    -7.754447e+00     2.376611e-01
 * time: 0.5463120937347412
     9    -7.804744e+00     8.952831e-02
 * time: 0.5689921379089355
    10    -7.829047e+00     9.353117e-02
 * time: 0.591832160949707
    11    -7.840757e+00     4.732102e-02
 * time: 0.6148121356964111
    12    -7.844583e+00     5.164201e-02
 * time: 0.6377201080322266
    13    -7.845133e+00     4.359320e-02
 * time: 0.66032910

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846818050889                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645906290                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.89s
  3   -7.852646686730      -13.29       -7.25    141ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.800942683154049e-7
|ρ_newton - ρ_scfv| = 2.4876312451115936e-7
|ρ_newton - ρ_dm|   = 8.95783771795882e-10
